# Koltsov3 Parameter Space Exploration (perm_type=1)

Systematically explore the Koltsov3 generator with **perm_type=1** across different coset groups.

## Search Space
- **k values**: 0, 1, 2, ... (where k+3 < n)
- **d values**: 1, 2, 3, ... (swap distance for S generator)
- **n values**: 4 to 30
- **coset types**: 5 groups (run separately)

## Koltsov3 Generators (perm_type=1)
- **I**: Swaps even pairs (0,1), (2,3), (4,5), ...
- **K**: Swaps odd pairs (1,2), (3,4), (5,6), ...
- **S**: Swaps positions k and k+d (configurable)

## Coset Groups
1. `different` - 2Different, 3Different, 4Different
2. `then` - Binary0then1, 0then1then2, etc.
3. `coincide` - 2Coincide, 3Coincide, etc.
4. `repeats` - Binary01Repeats, 012Repeats, etc.
5. `full_graph` - No coset (max_n=12)

## Cell 1: Imports and Setup

In [28]:
try:
    import numba
    # Use for CPU,GPU Kaggle machines:
    print('Install CayleyPy without dependencies (for Kaggle CPU,GPU):'); print()
    !pip install git+https://github.com/cayleypy/cayleypy --no-deps -q
except:
    print('Install CayleyPy with dependencies (for Kaggle TPU):'); print()
    # Use for TPU kaggle machines - since no numba
    !pip install git+https://github.com/cayleypy/cayleypy -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import os
from datetime import datetime
from tqdm.auto import tqdm
from cayleypy import CayleyGraph, PermutationGroups

print("Imports successful!")

Install CayleyPy without dependencies (for Kaggle CPU,GPU):

Imports successful!


## Cell 2: Configuration

In [29]:
PERM_TYPE = 1
MIN_N = 4
MAX_N = 30
MAX_D = 10  # Maximum value of d to explore
OUTPUT_DIR = "results_perm1"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Config: perm_type={PERM_TYPE}, n=[{MIN_N}, {MAX_N}], max_d={MAX_D}")
print(f"Output directory: {OUTPUT_DIR}")

Config: perm_type=1, n=[4, 30], max_d=10
Output directory: results_perm1


## Cell 3: Coset Group Definitions

In [30]:
COSET_GROUPS = {
    # =========================================================================
    # FULL GRAPH - No coset, explores entire permutation space
    # =========================================================================
    "full_graph": {
        # n=6: None (uses identity, explores all 6! = 720 permutations)
        "FullGraph": lambda n: None,
    },
    
    # =========================================================================
    # DIFFERENT - First D-1 elements are distinct, rest are all the same
    # Pattern: [0, 1, ..., D-2, D-1, D-1, D-1, ...]
    # =========================================================================
    "different": {
        # n=6: [0, 1, 1, 1, 1, 1]
        #"2Different": lambda n: list(range(1)) + [1]*(n-1) if n >= 2 else None,
        # n=6: [0, 1, 2, 2, 2, 2]
        #"3Different": lambda n: list(range(2)) + [2]*(n-2) if n >= 3 else None,
        # n=6: [0, 1, 2, 3, 3, 3]
        #"4Different": lambda n: list(range(3)) + [3]*(n-3) if n >= 4 else None,

        # n=6: [0, 1, 2, 3, 4, 4]
        "5Different": lambda n: list(range(4)) + [4]*(n-4) if n >= 5 else None,

        # n=7: [0, 1, 2, 3, 4, 5, 5]
        "6Different": lambda n: list(range(5)) + [5]*(n-5) if n >= 6 else None,

        # n=8: [0, 1, 2, 3, 4, 5, 6, 6]
        "7Different": lambda n: list(range(6)) + [6]*(n-6) if n >= 7 else None,

        # n=9: [0, 1, 2, 3, 4, 5, 6, 7, 7]
        "8Different": lambda n: list(range(7)) + [7]*(n-7) if n >= 8 else None,

    },
    
    # =========================================================================
    # THEN - Blocks of consecutive same values
    # Pattern: [0,0,..., 1,1,..., 2,2,...] (divided into equal-ish parts)
    # =========================================================================
    "then": {
        # n=6: [0, 0, 0, 1, 1, 1]
        "Binary0then1": lambda n: [0]*(n//2) + [1]*(n - n//2),
        # n=6: [0, 0, 1, 1, 2, 2]
        "0then1then2": lambda n: [0]*(n//3) + [1]*(n//3) + [2]*(n - 2*(n//3)),
        # n=8: [0, 0, 1, 1, 2, 2, 3, 3]
        "0then1then2then3": lambda n: [0]*(n//4) + [1]*(n//4) + [2]*(n//4) + [3]*(n - 3*(n//4)),
        # n=10: [0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
        "0then1then2then3then4": lambda n: [0]*(n//5) + [1]*(n//5) + [2]*(n//5) + [3]*(n//5) + [4]*(n - 4*(n//5)),
    },
    
    # =========================================================================
    # COINCIDE - Sequential values, but last C elements are all the same
    # Pattern: [0, 1, 2, ..., n-C-1, n-C, n-C, ..., n-C]
    # =========================================================================
    "coincide": {
        # n=6: [0, 1, 2, 3, 4, 4]
        "2Coincide": lambda n: list(range(n-2)) + [n-2]*2 if n > 2 else None,
        # n=6: [0, 1, 2, 3, 3, 3]
        "3Coincide": lambda n: list(range(n-3)) + [n-3]*3 if n > 3 else None,
        # n=6: [0, 1, 2, 2, 2, 2]
        "4Coincide": lambda n: list(range(n-4)) + [n-4]*4 if n > 4 else None,
        # n=6: [0, 1, 1, 1, 1, 1]
        "5Coincide": lambda n: list(range(n-5)) + [n-5]*5 if n > 5 else None,
        # n=7: [0, 1, 1, 1, 1, 1, 1]
        "6Coincide": lambda n: list(range(n-6)) + [n-6]*6 if n > 6 else None,
    },
    
    # =========================================================================
    # REPEATS - Repeating pattern of a short block
    # Pattern: [block] * (n // len(block)) + partial
    # =========================================================================
    "repeats": {
        # n=6: [0, 1, 0, 1, 0, 1]
        # n=7: [0, 1, 0, 1, 0, 1, 0]
        "Binary01Repeats": lambda n: [0,1]*(n//2) + [0]*(n - 2*(n//2)),
        # n=6: [0, 1, 0, 1, 0, 1]
        # n=7: [0, 1, 0, 1, 0, 1, 1]
        "Binary01Repeats_1": lambda n: [0,1]*(n//2) + [1]*(n - 2*(n//2)),
        # n=6: [0, 1, 2, 0, 1, 2]
        # n=7: [0, 1, 2, 0, 1, 2, 0]
        "012Repeats": lambda n: [0,1,2]*(n//3) + [0,1,2][:(n%3)],
        # n=6: [0, 1, 1, 0, 1, 1]
        # n=7: [0, 1, 1, 0, 1, 1, 0]
        "011Repeats": lambda n: [0,1,1]*(n//3) + [0,1,1][:(n%3)],
    },
}

# Print summary with examples
print("Coset Groups Summary:")
print("=" * 60)
for group_name, cosets in COSET_GROUPS.items():
    print(f"\n{group_name.upper()}:")
    for coset_name, func in cosets.items():
        example = func(9) if func(9) is not None else "None (full graph)"
        print(f"  {coset_name}: n=9 -> {example}")

Coset Groups Summary:

FULL_GRAPH:
  FullGraph: n=9 -> None (full graph)

DIFFERENT:
  5Different: n=9 -> [0, 1, 2, 3, 4, 4, 4, 4, 4]
  6Different: n=9 -> [0, 1, 2, 3, 4, 5, 5, 5, 5]
  7Different: n=9 -> [0, 1, 2, 3, 4, 5, 6, 6, 6]
  8Different: n=9 -> [0, 1, 2, 3, 4, 5, 6, 7, 7]

THEN:
  Binary0then1: n=9 -> [0, 0, 0, 0, 1, 1, 1, 1, 1]
  0then1then2: n=9 -> [0, 0, 0, 1, 1, 1, 2, 2, 2]
  0then1then2then3: n=9 -> [0, 0, 1, 1, 2, 2, 3, 3, 3]
  0then1then2then3then4: n=9 -> [0, 1, 2, 3, 4, 4, 4, 4, 4]

COINCIDE:
  2Coincide: n=9 -> [0, 1, 2, 3, 4, 5, 6, 7, 7]
  3Coincide: n=9 -> [0, 1, 2, 3, 4, 5, 6, 6, 6]
  4Coincide: n=9 -> [0, 1, 2, 3, 4, 5, 5, 5, 5]
  5Coincide: n=9 -> [0, 1, 2, 3, 4, 4, 4, 4, 4]
  6Coincide: n=9 -> [0, 1, 2, 3, 3, 3, 3, 3, 3]

REPEATS:
  Binary01Repeats: n=9 -> [0, 1, 0, 1, 0, 1, 0, 1, 0]
  Binary01Repeats_1: n=9 -> [0, 1, 0, 1, 0, 1, 0, 1, 1]
  012Repeats: n=9 -> [0, 1, 2, 0, 1, 2, 0, 1, 2]
  011Repeats: n=9 -> [0, 1, 1, 0, 1, 1, 0, 1, 1]


## Cell 4: Helper Functions

In [26]:
def is_valid_central(central):
    """Checks if central has >1 unique value."""
    return central is not None and len(np.unique(central)) > 1

def is_valid_params(n, k, d):
    """Check if (n, k, d) is a valid parameter combination for perm_type=1.
    
    Constraints:
    - k + d < n (S generator swaps positions k and k+d, both must be valid)
    - k >= 0
    - d >= 1
    """
    return k >= 0 and d >= 1 and k + d < n

def run_single_experiment(n, k, d, coset_name, coset_func):
    """Runs BFS for given parameters, returns BfsResult or None."""
    try:
        defn = PermutationGroups.koltsov3(n, perm_type=PERM_TYPE, k=k, d=d)

        central = coset_func(n)
        if coset_name != "FullGraph":
            if not is_valid_central(central):
                return None
            defn = defn.with_central_state(central)

        graph = CayleyGraph(defn)
        return graph.bfs(return_all_edges=False, return_all_hashes=False)
    except Exception as e:
        print(f"Error: {coset_name}, k={k}, d={d}, n={n}: {e}")
        return None

def get_group_dir(group_name):
    """Get the output directory for a group, creating it if needed."""
    group_dir = f"{OUTPUT_DIR}/{group_name}"
    os.makedirs(group_dir, exist_ok=True)
    return group_dir

def get_computed_combinations(group_name):
    """Return set of (coset, d, k, n) tuples already computed."""
    group_dir = get_group_dir(group_name)
    csv_path = f"{group_dir}/data.csv"

    if not os.path.exists(csv_path):
        return set()

    df = pd.read_csv(csv_path)
    return set(zip(df['coset'], df['d'], df['k'], df['n']))

def run_group(group_name, cosets, min_n=MIN_N, max_n=MAX_N, k_range=None, d_range=None, coset_filter=None, skip_computed=True):
    """
    Run experiments for a group of cosets, varying k and d.

    Args:
        group_name: Name of coset group
        cosets: Dictionary of coset functions
        min_n: Minimum n value (default: MIN_N)
        max_n: Maximum n value (default: MAX_N)
        k_range: Optional tuple (k_min, k_max); if None, k ranges 0 to max_n-2
        d_range: Optional tuple (d_min, d_max); if None, d ranges 1 to MAX_D
        coset_filter: Filter which cosets to run:
            - None: run all cosets in the group (default)
            - str: run only that coset (e.g., "3Different")
            - list: run only those cosets (e.g., ["2Different", "4Different"])
        skip_computed: If True, skip (coset, d, k, n) combinations already in CSV

    Returns:
        Dictionary with results for new computations only
    """
    computed = get_computed_combinations(group_name) if skip_computed else set()

    # Filter cosets based on coset_filter
    if coset_filter is None:
        filtered_cosets = cosets
    elif isinstance(coset_filter, str):
        if coset_filter not in cosets:
            raise ValueError(f"Coset '{coset_filter}' not found in group. Available: {list(cosets.keys())}")
        filtered_cosets = {coset_filter: cosets[coset_filter]}
    else:  # list
        filtered_cosets = {k: v for k, v in cosets.items() if k in coset_filter}
        missing = set(coset_filter) - set(filtered_cosets.keys())
        if missing:
            raise ValueError(f"Cosets not found: {missing}. Available: {list(cosets.keys())}")

    if k_range is None:
        k_values = list(range(max_n))
    else:
        k_values = list(range(k_range[0], k_range[1] + 1))

    if d_range is None:
        d_values = list(range(1, MAX_D + 1))
    else:
        d_values = list(range(d_range[0], d_range[1] + 1))

    results = {
        "metadata": {
            "perm_type": PERM_TYPE,
            "group": group_name,
            "timestamp": datetime.now().isoformat(),
            "n_range": [min_n, max_n],
            "k_range": list(k_range) if k_range else None,
            "d_range": list(d_range) if d_range else None,
            "coset_filter": coset_filter
        },
        "results": {}
    }

    total_new = 0

    for coset_name, coset_func in filtered_cosets.items():
        skipped = 0
        computed_count = 0
        results["results"][coset_name] = {}

        # Create progress bar for this coset
        total_iterations = len(d_values) * len(k_values) * (max_n - min_n + 1)
        pbar = tqdm(total=total_iterations, 
                    desc=f"{coset_name}", 
                    leave=True)

        for d in d_values:
            d_key = f"d={d}"
            results["results"][coset_name][d_key] = {}
            
            for k in k_values:
                k_key = f"k={k}"
                results["results"][coset_name][d_key][k_key] = {}

                for n in range(min_n, max_n + 1):
                    pbar.set_postfix({"d": d, "k": k, "n": n})
                    pbar.update(1)
                    
                    if not is_valid_params(n, k, d):
                        continue

                    # Skip if already computed
                    if (coset_name, d, k, n) in computed:
                        skipped += 1
                        continue

                    result = run_single_experiment(n, k, d, coset_name, coset_func)
                    if result is not None:
                        results["results"][coset_name][d_key][k_key][f"n={n}"] = {
                            "diameter": result.diameter(),
                            "growth": result.layer_sizes,
                            "last_layer_size": len(result.last_layer())
                        }
                        computed_count += 1

        pbar.close()
        total_new += computed_count
        print(f"  {coset_name}: Skipped {skipped} cached, computed {computed_count} new")

    print(f"Completed {group_name} ({total_new} new results)")
    return results

def save_results(group_name, results, cosets, append=True):
    """
    Save results to CSV (growth and central stored as JSON strings).

    Args:
        group_name: Name of coset group
        results: Results dictionary from run_group()
        cosets: Dictionary of coset functions (to compute central states)
        append: If True, append to existing CSV; if False, overwrite

    Returns:
        DataFrame with all results (existing + new)
    """
    group_dir = get_group_dir(group_name)
    csv_path = f"{group_dir}/data.csv"

    # Build DataFrame from new results
    rows = []
    for coset, d_data in results["results"].items():
        for d_key, k_data in d_data.items():
            d_val = int(d_key.split("=")[1])
            for k_key, n_data in k_data.items():
                k_val = int(k_key.split("=")[1])
                for n_key, metrics in n_data.items():
                    n_val = int(n_key.split("=")[1])
                    rows.append({
                        "coset": coset,
                        "d": d_val,
                        "k": k_val,
                        "n": n_val,
                        "diameter": metrics["diameter"],
                        "last_layer_size": metrics["last_layer_size"],
                        "total_states": sum(metrics["growth"]),
                        "growth": json.dumps(metrics["growth"])
                    })

    df_new = pd.DataFrame(rows)

    if append and os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        df = pd.concat([df_existing, df_new], ignore_index=True)
        # Remove duplicates, keeping last (new) values
        df = df.drop_duplicates(subset=['coset', 'd', 'k', 'n'], keep='last')
    else:
        df = df_new

    # Compute central states for all rows (backfills existing rows too)
    def compute_central(row):
        coset_func = cosets.get(row['coset'])
        if coset_func is None:
            return None
        central = coset_func(int(row['n']))
        return json.dumps(central) if central is not None else None
    
    df['central'] = df.apply(compute_central, axis=1)

    # Sort and save
    df = df.sort_values(['coset', 'd', 'k', 'n']).reset_index(drop=True)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path} ({len(df)} total rows, {len(df_new)} new)")
    return df

def load_results(group_name):
    """Load results from CSV, parsing growth back to list."""
    group_dir = get_group_dir(group_name)
    df = pd.read_csv(f"{group_dir}/data.csv")
    df['growth'] = df['growth'].apply(json.loads)
    if 'central' in df.columns:
        df['central'] = df['central'].apply(lambda x: json.loads(x) if pd.notna(x) else None)
    return df

def plot_group_results(group_name, df):
    """Create interactive Plotly plots for diameter, growth, and last layer size."""
    group_dir = get_group_dir(group_name)
    coset_names = list(df['coset'].unique())
    d_values = sorted(df['d'].unique())
    k_values = sorted(df['k'].unique())

    # Pre-parse growth column
    df = df.copy()
    df['growth_parsed'] = df['growth'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

    # ===== PLOT 1: Diameter vs n (dropdown for coset+d, lines for k) =====
    fig1 = go.Figure()
    trace_idx = 0
    trace_map1 = {}
    
    for coset_name in coset_names:
        for d_val in d_values:
            trace_map1[(coset_name, d_val)] = []
            coset_d_df = df[(df['coset'] == coset_name) & (df['d'] == d_val)]
            for k_val in k_values:
                k_df = coset_d_df[coset_d_df['k'] == k_val].sort_values('n')
                if len(k_df) > 0:
                    fig1.add_trace(go.Scatter(
                        x=k_df['n'], y=k_df['diameter'],
                        mode='lines+markers', name=f'k={k_val}',
                        visible=(coset_name == coset_names[0] and d_val == d_values[0]),
                        hovertemplate='n=%{x}<br>diameter=%{y}<br>k=' + str(k_val) + f'<br>d={d_val}'
                    ))
                    trace_map1[(coset_name, d_val)].append(trace_idx)
                    trace_idx += 1

    total_traces1 = trace_idx
    buttons1 = []
    first_button = True
    for coset_name in coset_names:
        for d_val in d_values:
            if not trace_map1.get((coset_name, d_val), []):
                continue
            visibility = [False] * total_traces1
            for idx in trace_map1[(coset_name, d_val)]:
                visibility[idx] = True
            
            # Calculate ranges for this (coset, d) combination
            coset_d_df = df[(df['coset'] == coset_name) & (df['d'] == d_val)]
            n_range = [coset_d_df['n'].min() - 0.5, coset_d_df['n'].max() + 0.5]
            diameter_range = [0, coset_d_df['diameter'].max() * 1.05]
            
            buttons1.append(dict(label=f'{coset_name}, d={d_val}', method='update', 
                                args=[{'visible': visibility},
                                      {'xaxis.range': n_range, 'yaxis.range': diameter_range}]))
            
            if first_button:
                init_n_range = n_range
                init_diameter_range = diameter_range
                first_button = False

    fig1.update_layout(
        title=dict(text=f'Diameter vs n - {group_name} (perm_type=1)', y=0.95),
        xaxis_title='n', yaxis_title='Diameter',
        xaxis=dict(range=init_n_range),
        yaxis=dict(range=init_diameter_range),
        updatemenus=[dict(buttons=buttons1, direction='down', x=0.0, xanchor='left', y=1.02, yanchor='bottom', showactive=True)],
        legend=dict(x=1.02, y=1), height=650, margin=dict(t=100)
    )
    fig1.write_html(f'{group_dir}/diameter.html')
    fig1.show()

    # ===== PLOT 2: Growth curves (dropdown for coset+d+k, lines for n) =====
    fig2 = go.Figure()
    trace_map2 = {}
    trace_idx = 0
    
    for coset_name in coset_names:
        for d_val in d_values:
            for k_val in k_values:
                trace_map2[(coset_name, d_val, k_val)] = []
                coset_d_k_df = df[(df['coset'] == coset_name) & (df['d'] == d_val) & (df['k'] == k_val)].sort_values('n')
                for _, row in coset_d_k_df.iterrows():
                    growth = row['growth_parsed']
                    fig2.add_trace(go.Scatter(
                        x=list(range(len(growth))), y=growth,
                        mode='lines+markers', name=f"n={row['n']}",
                        visible=(coset_name == coset_names[0] and d_val == d_values[0] and k_val == k_values[0]),
                        hovertemplate='distance=%{x}<br>layer_size=%{y}<br>n=' + str(row['n'])
                    ))
                    trace_map2[(coset_name, d_val, k_val)].append(trace_idx)
                    trace_idx += 1

    total_traces2 = trace_idx
    buttons2 = []
    first_button = True
    for coset_name in coset_names:
        for d_val in d_values:
            for k_val in k_values:
                if not trace_map2.get((coset_name, d_val, k_val), []):
                    continue
                visibility = [False] * total_traces2
                for idx in trace_map2[(coset_name, d_val, k_val)]:
                    visibility[idx] = True
                
                # Calculate ranges for this (coset, d, k) combination
                coset_d_k_df = df[(df['coset'] == coset_name) & (df['d'] == d_val) & (df['k'] == k_val)]
                growths = coset_d_k_df['growth_parsed'].tolist()
                if growths:
                    max_distance = max(len(g) for g in growths)
                    max_layer = max(max(g) for g in growths)
                    min_layer = min(min(g) for g in growths if min(g) > 0)
                    distance_range = [-0.5, max_distance + 0.5]
                    layer_range = [np.log10(min_layer * 0.5), np.log10(max_layer * 2)]
                else:
                    distance_range = [0, 10]
                    layer_range = [0, 6]
                
                buttons2.append(dict(label=f'{coset_name}, d={d_val}, k={k_val}', method='update', 
                                    args=[{'visible': visibility},
                                          {'xaxis.range': distance_range, 'yaxis.range': layer_range}]))
                
                if first_button:
                    init_distance_range = distance_range
                    init_layer_range = layer_range
                    first_button = False

    fig2.update_layout(
        title=dict(text=f'Growth Curves - {group_name} (perm_type=1)', y=0.95),
        xaxis_title='Distance', yaxis_title='Layer Size', yaxis_type='log',
        xaxis=dict(range=init_distance_range),
        yaxis=dict(range=init_layer_range),
        updatemenus=[dict(buttons=buttons2, direction='down', x=0.0, xanchor='left', y=1.02, yanchor='bottom', showactive=True)],
        legend=dict(x=1.02, y=1), height=650, margin=dict(t=100)
    )
    fig2.write_html(f'{group_dir}/growth.html')
    fig2.show()

    # ===== PLOT 3: Last layer size vs n (dropdown for coset+d, lines for k) =====
    fig3 = go.Figure()
    trace_idx = 0
    trace_map3 = {}
    
    for coset_name in coset_names:
        for d_val in d_values:
            trace_map3[(coset_name, d_val)] = []
            coset_d_df = df[(df['coset'] == coset_name) & (df['d'] == d_val)]
            for k_val in k_values:
                k_df = coset_d_df[coset_d_df['k'] == k_val].sort_values('n')
                if len(k_df) > 0:
                    fig3.add_trace(go.Scatter(
                        x=k_df['n'], y=k_df['last_layer_size'],
                        mode='lines+markers', name=f'k={k_val}',
                        visible=(coset_name == coset_names[0] and d_val == d_values[0]),
                        hovertemplate='n=%{x}<br>last_layer=%{y}<br>k=' + str(k_val) + f'<br>d={d_val}'
                    ))
                    trace_map3[(coset_name, d_val)].append(trace_idx)
                    trace_idx += 1

    total_traces3 = trace_idx
    buttons3 = []
    first_button = True
    for coset_name in coset_names:
        for d_val in d_values:
            if not trace_map3.get((coset_name, d_val), []):
                continue
            visibility = [False] * total_traces3
            for idx in trace_map3[(coset_name, d_val)]:
                visibility[idx] = True
            
            # Calculate ranges for this (coset, d) combination
            coset_d_df = df[(df['coset'] == coset_name) & (df['d'] == d_val)]
            n_range = [coset_d_df['n'].min() - 0.5, coset_d_df['n'].max() + 0.5]
            ll_min = coset_d_df['last_layer_size'].min()
            ll_max = coset_d_df['last_layer_size'].max()
            last_layer_range = [np.log10(ll_min * 0.5), np.log10(ll_max * 2)]
            
            buttons3.append(dict(label=f'{coset_name}, d={d_val}', method='update', 
                                args=[{'visible': visibility},
                                      {'xaxis.range': n_range, 'yaxis.range': last_layer_range}]))
            
            if first_button:
                init_n_range3 = n_range
                init_ll_range = last_layer_range
                first_button = False

    fig3.update_layout(
        title=dict(text=f'Last Layer Size vs n - {group_name} (perm_type=1)', y=0.95),
        xaxis_title='n', yaxis_title='Last Layer Size', yaxis_type='log',
        xaxis=dict(range=init_n_range3),
        yaxis=dict(range=init_ll_range),
        updatemenus=[dict(buttons=buttons3, direction='down', x=0.0, xanchor='left', y=1.02, yanchor='bottom', showactive=True)],
        legend=dict(x=1.02, y=1), height=650, margin=dict(t=100)
    )
    fig3.write_html(f'{group_dir}/lastlayer.html')
    fig3.show()
    
    print(f"Interactive plots saved to {group_dir}/")

print("Helper functions defined!")

Helper functions defined!


---
## Group 1: "different" (3 cosets)
- 2Different: [0, 1, 1, 1, ...]
- 3Different: [0, 1, 2, 2, ...]
- 4Different: [0, 1, 2, 3, 3, ...]

In [31]:
%%time
group_name = "different"

# Customize ranges as needed:
# - min_n, max_n: range of n values to compute
# - k_range: tuple (k_min, k_max) or None for all valid k
# - d_range: tuple (d_min, d_max) or None for d=1 to MAX_D
# - Results are cached: only new (coset, d, k, n) combinations are computed
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=9, max_n=20, k_range=(1,4), d_range=(1,5))
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

5Different:   5%|▍         | 11/240 [00:00<00:00, 1195.42it/s, d=1, k=1, n=20]

5Different: 100%|██████████| 240/240 [00:00<00:00, 1520.64it/s, d=5, k=4, n=20]


  5Different: Skipped 239 cached, computed 0 new


6Different: 100%|██████████| 240/240 [00:00<00:00, 1569.79it/s, d=5, k=4, n=20]


  6Different: Skipped 239 cached, computed 0 new


7Different: 100%|██████████| 240/240 [00:00<00:00, 1514.83it/s, d=5, k=4, n=20]


  7Different: Skipped 239 cached, computed 0 new


8Different: 100%|██████████| 240/240 [00:00<00:00, 1515.45it/s, d=5, k=4, n=20]


  8Different: Skipped 239 cached, computed 0 new
Completed different (0 new results)
Saved: results_perm1/different/data.csv (1673 total rows, 0 new)
CPU times: user 1 s, sys: 151 ms, total: 1.15 s
Wall time: 1.16 s


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

---
## Group 2: "then" (4 cosets)
- Binary0then1: [0,0,...,1,1,...]
- 0then1then2: [0,...,1,...,2,...]
- 0then1then2then3: quarters
- 0then1then2then3then4: fifths

In [6]:
%%time
group_name = "then"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None, d_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

CPU times: user 11 μs, sys: 0 ns, total: 11 μs
Wall time: 13.6 μs


NameError: name 'COSET_GROUPS' is not defined

---
## Group 3: "coincide" (5 cosets)
- 2Coincide: [..., x, x]
- 3Coincide: [..., x, x, x]
- 4Coincide, 5Coincide, 6Coincide

In [ ]:
%%time
group_name = "coincide"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None, d_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Group 4: "repeats" (4 cosets)
- Binary01Repeats: [0,1,0,1,...]
- Binary01Repeats_1: [0,1,0,1,...] with 1s at end
- 012Repeats: [0,1,2,0,1,2,...]
- 011Repeats: [0,1,1,0,1,1,...]

In [ ]:
%%time
group_name = "repeats"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None, d_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Group 5: "full_graph" (1 coset, max_n=12)
- FullGraph: No central state, explores entire permutation space
- **Warning**: n! grows very fast, so we limit to max_n=12

In [ ]:
%%time
group_name = "full_graph"

# Customize ranges as needed (max_n=12 recommended for full graph due to n! growth):
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=12, k_range=None, d_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Summary: List all output files

In [ ]:
import glob

print("Output files generated:")
for f in sorted(glob.glob(f"{OUTPUT_DIR}/*")):
    size = os.path.getsize(f)
    print(f"  {f} ({size:,} bytes)")

---
## Compare d values
Quick comparison of how d affects diameter for a specific coset

In [ ]:
def plot_d_comparison(df, coset_name, k_val=0):
    """Compare diameter vs n for different d values at fixed coset and k."""
    fig = go.Figure()
    
    subset = df[(df['coset'] == coset_name) & (df['k'] == k_val)]
    d_values = sorted(subset['d'].unique())
    
    for d_val in d_values:
        d_df = subset[subset['d'] == d_val].sort_values('n')
        if len(d_df) > 0:
            fig.add_trace(go.Scatter(
                x=d_df['n'], y=d_df['diameter'],
                mode='lines+markers', name=f'd={d_val}',
                hovertemplate='n=%{x}<br>diameter=%{y}<br>d=' + str(d_val)
            ))
    
    fig.update_layout(
        title=f'Diameter vs n for {coset_name} (k={k_val}) - Comparing d values',
        xaxis_title='n', yaxis_title='Diameter',
        legend=dict(x=1.02, y=1),
        height=500
    )
    fig.show()

# Example: Compare d values for 4Different coset with k=0
# Uncomment and modify as needed:
# plot_d_comparison(df_different, '4Different', k_val=0)